### CNN Model

CNN model structure based on Automatic sleep stages classification using multi-level fusion by H. Kim.

Labels come from 2 different PSG systems: Remlogic and Somnomedics. Sleep stage labels are as follows:

0 = Awake = SLEEP-S0 \
1 = REM = SLEEP-REM \
2 = Light = SLEEP-S1 and SLEEP-S2 \
3 = Deep = SLEEP-S3

EMFIT is the piezoelectric mat and Somnofy is the radar.

In [2]:
! pip3 install tensorflow

     |████████████████████████████████| 244.3 MB 6.2 kB/s eta 0:00:012
     |████████████████████████████████| 1.7 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 439 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 1.6 MB/s eta 0:00:01


In [106]:
from tensorflow.keras.layers import Dense, Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [8]:
import os
from helpers import *
from constants import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [80]:
subjectID = PARTICIPANT_IDS[0]
sleep_stages, nan_counts = impute_data(subjectID)
labels = sleep_stages["sleep_stage_num_psg"]
features = sleep_stages.drop(columns="sleep_stage_num_psg")
size = sleep_stages.shape[0]
divide_ind = int(size*.7)
x_train, y_train = features[:divide_ind].reset_index(drop=True), labels[:divide_ind].reset_index(drop=True)
x_train, y_train = tf.expand_dims(x_train, 0), tf.expand_dims(y_train, 0)
x_test, y_test = features[divide_ind:].reset_index(drop=True), labels[divide_ind:].reset_index(drop=True)
x_test, y_test = tf.expand_dims(x_test, 0), tf.expand_dims(y_test, 0)

In [63]:
x_train.shape

TensorShape([1, 667, 2])

In [101]:
cnn = Sequential()
cnn.add(Conv1D(filters=64,kernel_size=2,padding="same",activation="relu",input_shape=(divide_ind,2)))

adam = Adam(learning_rate=0.001)
cnn.compile(loss='mse', optimizer=adam)

In [114]:
# 64 filters, 10 kernel size
cnn = Sequential()
cnn.add(Conv1D(64, 10, activation='relu',input_shape=(divide_ind,2)))
cnn.add(MaxPooling1D())
cnn.add(BatchNormalization())

cnn.add(Conv1D(128, 10, activation='relu'))
cnn.add(MaxPooling1D())
cnn.add(BatchNormalization())

cnn.add(Conv1D(128, 10, activation='relu'))
cnn.add(MaxPooling1D())
cnn.add(BatchNormalization())

cnn.add(Conv1D(256, 10, activation='relu'))
cnn.add(MaxPooling1D())
cnn.add(BatchNormalization())

cnn.add(Flatten()) 
cnn.add(Dense(1024, activation='relu', name='dense_1'))
cnn.add(BatchNormalization())
cnn.add(Dropout(.2))

cnn.add(Dense(2048, activation='relu', name='dense_2'))
cnn.add(BatchNormalization())
cnn.add(Dropout(.2))

cnn.add(Dense(4, activation='softmax', name='predictions'))

adam = Adam(learning_rate=0.001)
cnn.compile(loss='mse', optimizer=adam)

In [115]:
cnn.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_42 (Conv1D)          (None, 658, 64)           1344      
                                                                 
 max_pooling1d_32 (MaxPoolin  (None, 329, 64)          0         
 g1D)                                                            
                                                                 
 batch_normalization_32 (Bat  (None, 329, 64)          256       
 chNormalization)                                                
                                                                 
 conv1d_43 (Conv1D)          (None, 320, 128)          82048     
                                                                 
 max_pooling1d_33 (MaxPoolin  (None, 160, 128)         0         
 g1D)                                                            
                                                     

In [116]:
cnn.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          verbose=2)

Epoch 1/15


ValueError: in user code:

    File "/Users/elifkurtay/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/elifkurtay/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/elifkurtay/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/elifkurtay/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/elifkurtay/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/Users/elifkurtay/opt/anaconda3/lib/python3.9/site-packages/keras/engine/compile

In [79]:
x_test.shape

(287, 2)

In [85]:
preds = cnn.predict(x_train)

1/1 [==============================] - 0s 18ms/step


In [88]:
preds

array([[0.00469779]], dtype=float32)

In [87]:
# Accuracy score 
print(accuracy_score(preds, y_train))

# Confusion Matrix and Classification Report 
print(confusion_matrix(preds, y_train))
print(classification_report(preds, y_train))

ValueError: Classification metrics can't handle a mix of continuous and multiclass-multioutput targets